## 영화 5개 받기

In [1]:
print("좋아하는 영화 5개를 입력해주세요(구분은 ,로): ")
movie_list = input().split(",")
print(movie_list)

좋아하는 영화 5개를 입력해주세요(구분은 ,로): 
모가디슈, 블랙위도우, 라라랜드, 천문, 크루엘라
['모가디슈', ' 블랙위도우', ' 라라랜드', ' 천문', ' 크루엘라']


## 영화 정보

In [2]:
import requests
import json

In [4]:
movie_cd_list = []
director_list = []
selected_movie_list = []

for movie in movie_list:
    movie_cd_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=d99e474ec53733a372d8413bd2753506&movieNm='+movie
    res = requests.get(movie_cd_url)
    text = res.text
    d = json.loads(text)
    if d['movieListResult']['totCnt'] > 1:    # 동명의 영화가 있을 경우
        item_year = input(movie+"의 개봉연도를 입력하세요: ")   # 개봉연도로 구분
        for item in d['movieListResult']['movieList']:
            if item_year == item['openDt'][:4]:
                # director_list = director['peopleNm']
                movie_cd_list.append([item['movieCd'],
                                      item['movieNm'],
                                      item['genreAlt'],
                                      item['openDt'][:4]])
                selected_movie_list.append(item['movieNm'])
    if d['movieListResult']['totCnt'] == 1:
        for item in d['movieListResult']['movieList']:
            movie_cd_list.append([item['movieCd'],
                                  item['movieNm'],
                                  item['genreAlt'],
                                  item['openDt'][:4]])
            selected_movie_list.append(item['movieNm'])

print(movie_cd_list)
print(selected_movie_list)

모가디슈의 개봉연도를 입력하세요: 2021
 블랙위도우의 개봉연도를 입력하세요: 2021
 천문의 개봉연도를 입력하세요: 2019
[['20204117', '모가디슈', '액션,드라마', '2021'], ['20191951', '블랙 위도우', '액션,어드벤처,SF', '2021'], ['20167904', '라라랜드', '드라마,뮤지컬,멜로/로맨스', '2016'], ['20184571', '천문: 하늘에 묻는다', '사극', '2019'], ['20216362', '크루엘라', '드라마,범죄,코미디', '2021']]
['모가디슈', '블랙 위도우', '라라랜드', '천문: 하늘에 묻는다', '크루엘라']


In [5]:
import pandas as pd

movie_cdlist_df = pd.DataFrame(movie_cd_list)
movie_cdlist_df.columns = ['movieCd', 'movieNm','genreAlt', 'openYr']
movie_cdlist_df

,movieCd,movieNm,genreAlt,openYr
0,20204117,모가디슈,"액션,드라마",2021
1,20191951,블랙 위도우,"액션,어드벤처,SF",2021
2,20167904,라라랜드,"드라마,뮤지컬,멜로/로맨스",2016
3,20184571,천문: 하늘에 묻는다,사극,2019
4,20216362,크루엘라,"드라마,범죄,코미디",2021


## 영화코드로 정보 추출

In [6]:
movie_detail_list = []
movie_director_list = []
movie_cast_list = []    # 한 영화당 최대 10명의 배우를 추출하여 저장한 리스트
direct_list = []
movie_mainactor_list = []    # 한 영화당 최대 2명(주인공)의 배우를 추출하여 저장한 리스트
movie_genre_list = [] 

actor_list = []              # 5개 영화의 movie_cast_list을 모은 것
mainactor_list = []          # 5개 영화의 movie_mainactor_list을 모은 것
genre_list = []              # 5개 영화의 장르를 모은 것

movie_actor_list = [[] for i in range(0,5)]
movie_direc_list = [[] for i in range(0,5)]
j=0

for movieCd in movie_cdlist_df['movieCd']:
    movie_director_list.clear()
    movie_cast_list.clear()
    movie_mainactor_list.clear()
    movie_genre_list.clear()
    actor_list_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=d99e474ec53733a372d8413bd2753506&movieCd='+movieCd
    res = requests.get(actor_list_url)
    text = res.text
    d = json.loads(text)

    for direc in d['movieInfoResult']['movieInfo']['directors']:
        movie_director_list.append(direc['peopleNm'])
        direct_list+=movie_director_list

    for cast in d['movieInfoResult']['movieInfo']['actors']:
        if len(movie_mainactor_list) < 3:
            movie_mainactor_list.append(cast['peopleNm'])
        if len(movie_cast_list) <= 9:
            movie_cast_list.append(cast['peopleNm'])
            
    for genr in d['movieInfoResult']['movieInfo']['genres']:
        movie_genre_list.append(genr['genreNm'])
  
    #print(movie_cast_list)
    actor_list += movie_cast_list
    mainactor_list += movie_mainactor_list
    genre_list += movie_genre_list

    """for i in range(0,5):
        movie_direc_list[i] = direct_list[i:i+1]
        movie_actor_list[i] = actor_list[10*i:10*(i+1)]"""

    """movie_detail_list.append([d['movieInfoResult']['movieInfo']['movieCd'],
                              d['movieInfoResult']['movieInfo']['movieNm'],
                              d['movieInfoResult']['movieInfo']['openDt'][:4],
                              movie_direc_list[j], movie_actor_list[j]])"""
    j+=1

print(direct_list)
print(actor_list)
print(mainactor_list)
print(genre_list)

['류승완', '케이트 쇼트랜드', '데이미언 셔젤', '허진호', '크레이그 질레스피']
['김윤석', '조인성', '허준호', '구교환', '김소진', '정만식', '김재화', '박경혜', '박명신', '한철우', '스칼렛 요한슨', '플로렌스 퓨', '레이첼 와이즈', '데이빗 하버', '엠마 스톤', '라이언 고슬링', '핀 위트록', 'J.K. 시몬스', '로즈마리 드윗', '최민식', '한석규', '신구', '김홍파', '허준호', '김태우', '김원해', '임원희', '오광록', '박성훈', '엠마 스톤', '엠마 톰슨', '마크 스트롱']
['김윤석', '조인성', '허준호', '스칼렛 요한슨', '플로렌스 퓨', '레이첼 와이즈', '엠마 스톤', '라이언 고슬링', '핀 위트록', '최민식', '한석규', '신구', '엠마 스톤', '엠마 톰슨', '마크 스트롱']
['액션', '드라마', '액션', '어드벤처', 'SF', '드라마', '뮤지컬', '멜로/로맨스', '사극', '드라마', '범죄', '코미디']


## 배우 등장 횟수

In [7]:
actor_dict = dict()
for i in actor_list:
    if i not in actor_dict:
        actor_dict[i]=1
    else:
        actor_dict[i]=actor_dict[i]+1

actor_dict

{'김윤석': 1,
 '조인성': 1,
 '허준호': 2,
 '구교환': 1,
 '김소진': 1,
 '정만식': 1,
 '김재화': 1,
 '박경혜': 1,
 '박명신': 1,
 '한철우': 1,
 '스칼렛 요한슨': 1,
 '플로렌스 퓨': 1,
 '레이첼 와이즈': 1,
 '데이빗 하버': 1,
 '엠마 스톤': 2,
 '라이언 고슬링': 1,
 '핀 위트록': 1,
 'J.K. 시몬스': 1,
 '로즈마리 드윗': 1,
 '최민식': 1,
 '한석규': 1,
 '신구': 1,
 '김홍파': 1,
 '김태우': 1,
 '김원해': 1,
 '임원희': 1,
 '오광록': 1,
 '박성훈': 1,
 '엠마 톰슨': 1,
 '마크 스트롱': 1}

## 주연배우(맨 앞 3사람)

In [8]:
mainactor_dict = dict()
for i in mainactor_list:
    if i not in mainactor_dict:
        mainactor_dict[i]=1
    else:
        mainactor_dict[i]=mainactor_dict[i]+1

mainactor_dict

{'김윤석': 1,
 '조인성': 1,
 '허준호': 1,
 '스칼렛 요한슨': 1,
 '플로렌스 퓨': 1,
 '레이첼 와이즈': 1,
 '엠마 스톤': 2,
 '라이언 고슬링': 1,
 '핀 위트록': 1,
 '최민식': 1,
 '한석규': 1,
 '신구': 1,
 '엠마 톰슨': 1,
 '마크 스트롱': 1}

## 추천배우 5명

- 2번 등장 배우 우선
- 빈자리는 랜덤

In [9]:
import random

prefer_actor_list = []
for key, value in actor_dict.items():
    if value > 1:
        prefer_actor_list.append(key)
if len(prefer_actor_list)<5:
    for i in range(5-len(prefer_actor_list)):
        actor = random.choice(mainactor_list)
        if actor not in prefer_actor_list:
            prefer_actor_list.append(actor)

prefer_actor_list

['허준호', '엠마 스톤', '김윤석', '라이언 고슬링']

## 배우 추천 코드

In [10]:
actor_code = []
filmo_list = []
final_filmo_list = []

for prefer_actor in prefer_actor_list:
    actor_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key=d99e474ec53733a372d8413bd2753506'+'&peopleNm='+prefer_actor
    res = requests.get(actor_list_url)
    text = res.text
    d = json.loads(text)

    for actor in d['peopleListResult']['peopleList']:
        final_filmo_list.clear()
        if actor['repRoleNm'] == '배우' and actor['peopleNmEn']!= "":
            filmo_list = actor['filmoNames'].split("|")
            for filmo in filmo_list:
                if filmo not in selected_movie_list:
                    final_filmo_list.append(filmo)
            actor_code.append([actor['peopleCd'], actor['peopleNm'],actor['peopleNmEn'], actor['repRoleNm'], final_filmo_list[:5]])

In [11]:
print(selected_movie_list)
print(filmo_list)
print(final_filmo_list)

['모가디슈', '블랙 위도우', '라라랜드', '천문: 하늘에 묻는다', '크루엘라']
['퍼스트맨', '블레이드 러너 2049', '송 투 송', '라라랜드', '나이스 가이즈', '빅쇼트', '로스트리버', '온리 갓 포기브스', '플레이스 비욘드 더 파인즈', '갱스터 스쿼드', '킹메이커', '드라이브', '크레이지 스투피드 러브', '블루 발렌타인', '올 굿 에브리씽', '내겐 너무 사랑스러운 그녀', '프랙처', '하프넬슨', '노트북', '유나이티드 스테이츠 오브 리랜드', '빌리버']
['퍼스트맨', '블레이드 러너 2049', '송 투 송', '나이스 가이즈', '빅쇼트', '로스트리버', '온리 갓 포기브스', '플레이스 비욘드 더 파인즈', '갱스터 스쿼드', '킹메이커', '드라이브', '크레이지 스투피드 러브', '블루 발렌타인', '올 굿 에브리씽', '내겐 너무 사랑스러운 그녀', '프랙처', '하프넬슨', '노트북', '유나이티드 스테이츠 오브 리랜드', '빌리버']


In [13]:
actor_code_df = pd.DataFrame(actor_code)
actor_code_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm', 'filmoNames']
actor_code_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,10088453,허준호,HUH Joon-ho,배우,"[노량: 죽음의 바다, 결백, 퍼펙트맨, 국가부도의 날, 인랑]"
1,10049633,엠마 스톤,Emma Stone,배우,"[크루즈 패밀리: 뉴 에이지, 좀비랜드: 더블 탭, 더 페이버릿: 여왕의 여자, 빌..."
2,10005276,김윤석,KIM Yun-seok,배우,"[노량: 죽음의 바다, 바이러스(가제), 미성년, 암수살인, 1987]"
3,10014012,라이언 고슬링,Ryan Gosling,배우,"[퍼스트맨, 블레이드 러너 2049, 송 투 송, 나이스 가이즈, 빅쇼트]"


## 장르 등장 횟수 

In [14]:
genre_dict = dict()
for i in genre_list:
    if i not in genre_dict:
        genre_dict[i]=1
    else:
        genre_dict[i]=genre_dict[i]+1

genre_dict

{'액션': 2,
 '드라마': 3,
 '어드벤처': 1,
 'SF': 1,
 '뮤지컬': 1,
 '멜로/로맨스': 1,
 '사극': 1,
 '범죄': 1,
 '코미디': 1}

In [70]:
genre_df = pd.DataFrame(list(genre_dict.items()), columns=['genres', 'number'])
genre_df

,genres,number
0,액션,2
1,드라마,3
2,어드벤처,1
3,SF,1
4,뮤지컬,1
5,멜로/로맨스,1
6,사극,1
7,범죄,1
8,코미디,1


In [121]:
recom_genre=[]
recom_genre=genre_df[genre_df['number']>1]
recom_genre_list=recom_genre['genres']
recom_genre_list

0     액션
1    드라마
Name: genres, dtype: object

## 추려진 영화 장르 검색

In [122]:
genre_search=[]

for i in range(len(final_filmo_list)):
    genre_search_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=d99e474ec53733a372d8413bd2753506'+'&movieNm='+final_filmo_list[i]
    res = requests.get(genre_search_url)
    text = res.text
    d = json.loads(text)
        
    for movie in d['movieListResult']['movieList']:
        genre_search.append([movie['movieNm'], movie['genreAlt']])

print(genre_search)

[['퍼스트맨', 'SF,드라마'], ['블레이드 러너 2049', 'SF,액션'], ['송 투 송', '드라마,멜로/로맨스'], ['나이스 가이즈', '범죄,미스터리,스릴러'], ['빅쇼트', '드라마'], ['로스트리버', '판타지,스릴러'], ['로스트 리버 - 링컨의 비밀병기', '사극'], ['온리 갓 포기브스', '액션'], ['플레이스 비욘드 더 파인즈', '범죄,드라마'], ['갱스터 스쿼드', '범죄,드라마'], ['킹메이커', '드라마'], ['킹메이커', '드라마'], ['킹메이커', '드라마'], ['드라이브(가제)', '스릴러'], ['드라이브 마이 카', '드라마'], ['오버 드라이브', '액션'], ['드라이브: 분노의 질주', '스릴러'], ['본 투 드라이브 ', '다큐멘터리'], ['드라이브웨이', '기타'], ['드라이브', '액션,멜로/로맨스'], ['EXiS2020 인디비주얼 2 + 4 + 5', '기타'], ['쉬머 박스 드라이브', '기타'], ['판타스틱 전기자동차 드라이브', '애니메이션'], ['크레이지 스투피드 러브', '코미디,멜로/로맨스,드라마'], ['블루 발렌타인', '드라마,멜로/로맨스'], ['올 굿 에브리씽', '미스터리,드라마'], ['내겐 너무 사랑스러운 그녀', '코미디,드라마'], ['프랙처', '범죄,미스터리'], ['하프넬슨', '드라마'], ['노트북', '멜로/로맨스'], ['유나이티드 스테이츠 오브 리랜드', '범죄,드라마'], ['언빌리버블 스페이스 러브', '애니메이션'], ['빌리버', '드라마'], ['빌리버스', '공포(호러),스릴러'], ['빌리버스', '']]


In [201]:
pre_recom_pd=pd.DataFrame(genre_search)
pre_recom_pd.columns=['movieNm', 'genres']
pre_recom_pd

,movieNm,genres
0,퍼스트맨,"SF,드라마"
1,블레이드 러너 2049,"SF,액션"
2,송 투 송,"드라마,멜로/로맨스"
3,나이스 가이즈,"범죄,미스터리,스릴러"
4,빅쇼트,드라마
5,로스트리버,"판타지,스릴러"
6,로스트 리버 - 링컨의 비밀병기,사극
7,온리 갓 포기브스,액션
8,플레이스 비욘드 더 파인즈,"범죄,드라마"
9,갱스터 스쿼드,"범죄,드라마"


In [216]:
pre_recom_genre=[]
for i in range(0, len(recom_genre_list)):
    pre_recom_genre.append(pre_recom_pd[pre_recom_pd['genres']==recom_genre_list[i]]['movieNm'])
pre_recom_genre

[7     온리 갓 포기브스
 15      오버 드라이브
 Name: movieNm, dtype: object,
 4           빅쇼트
 10         킹메이커
 11         킹메이커
 12         킹메이커
 14    드라이브 마이 카
 28         하프넬슨
 32          빌리버
 Name: movieNm, dtype: object]

In [244]:
recom_df=pd.DataFrame(pre_recom_genre)
recom_df

,4,7,10,11,12,14,15,28,32
movieNm,NaN,온리 갓 포기브스,NaN,NaN,NaN,NaN,오버 드라이브,NaN,NaN
movieNm,빅쇼트,NaN,킹메이커,킹메이커,킹메이커,드라이브 마이 카,NaN,하프넬슨,빌리버


In [245]:
recom_df=recom_df.transpose()
recom_df

,movieNm,movieNm
4,NaN,빅쇼트
7,온리 갓 포기브스,NaN
10,NaN,킹메이커
11,NaN,킹메이커
12,NaN,킹메이커
14,NaN,드라이브 마이 카
15,오버 드라이브,NaN
28,NaN,하프넬슨
32,NaN,빌리버


In [265]:
recom_df_reset=recom_df.reset_index()
del recom_df_reset['index']
recom_df_reset

,movieNm,movieNm
0,NaN,빅쇼트
1,온리 갓 포기브스,NaN
2,NaN,킹메이커
3,NaN,킹메이커
4,NaN,킹메이커
5,NaN,드라이브 마이 카
6,오버 드라이브,NaN
7,NaN,하프넬슨
8,NaN,빌리버


In [283]:
recommendation=[]
null_recom = recom_df_reset.notnull()
null_recom
#if recom_df_reset['movieNm'] == recom_df_reset.notnull():
#    print(recom_df_reset)
#recommendation = recom_df_reset[recom_df_reset.isnull()==False]['movieNm']
#    recom_df_reset = recom_df_reset
#if recom_df.notnull() == True:
#recommendation
#recommendation

,movieNm,movieNm
0,False,True
1,True,False
2,False,True
3,False,True
4,False,True
5,False,True
6,True,False
7,False,True
8,False,True


In [247]:
recom_list=[]
recom_list=recom_df.values.tolist()
recom_list

[[nan, '빅쇼트'],
 ['온리 갓 포기브스', nan],
 [nan, '킹메이커'],
 [nan, '킹메이커'],
 [nan, '킹메이커'],
 [nan, '드라이브 마이 카'],
 ['오버 드라이브', nan],
 [nan, '하프넬슨'],
 [nan, '빌리버']]

In [248]:
print(recom_list[:])

[[nan, '빅쇼트'], ['온리 갓 포기브스', nan], [nan, '킹메이커'], [nan, '킹메이커'], [nan, '킹메이커'], [nan, '드라이브 마이 카'], ['오버 드라이브', nan], [nan, '하프넬슨'], [nan, '빌리버']]


In [ ]:
newList = list(filter(isBiggerThanFive, originalList))